In [1]:
import pandas as pd
import requests
import wget

import folium
from folium import plugins
from folium.plugins import MarkerCluster

In [2]:
df0 = pd.read_csv("glasgow_bakery_addresses.csv")
df0.head()

,bakery_name,addres
0,Milo Bakes,"1054 Cathcart Rd, Mount Florida, Glasgow G42 9XW"
1,Full Circle Bakes,"276 Cumbernauld Rd, Glasgow G31 2UL"
2,Lily Bakes Cakes,"31 Hyndland St, Partick, Glasgow G11 5QF"
3,Big Bear Bakery,"21 Dowanhill St, Partick, Glasgow G11 5QR"
4,Long Short Black,"501 Victoria Rd, Glasgow G42 8RL"


In [3]:
def query_address(address):
    """Return response from open streetmap.
    
    Parameter:
    address - address of establishment
    
    Returns:
    result - json, response from open street map
    """
    
    url = "https://nominatim.openstreetmap.org/search"
    parameters = {'q':'{}, Scotland'.format(address), 'format':'json'}
    
    response = requests.get(url, params=parameters)
    # don't want to raise an error to not stop the processing
    # print address instead for future inspection
    if response.status_code != 200:
        print("Error querying {}".format(address))
        result = {}
    else:
        result = response.json()
    return result

In [4]:
df0['json'] = df0['addres'].map(lambda x: query_address(x))

# drop empty responses
df1 = df0[df0['json'].map(lambda d: len(d)) > 0].copy()
print(df1.shape[0])

11


In [5]:
# extract relevant fields from API response (json format)
df1['lat'] = df1['json'].map(lambda x: x[0]['lat'])
df1['lon'] = df1['json'].map(lambda x: x[0]['lon'])
df1['type'] = df1['json'].map(lambda x: x[0]['type'])

# for name in df1.iterrows():
#     df1['geometry'] = [float(name[1]['lat']), float(name[1]['lon'])]

df1['geometry'] =  list(zip(df1['lat'], df1['lon']))

In [6]:
df1

,bakery_name,addres,json,lat,lon,type,geometry
0,Milo Bakes,"1054 Cathcart Rd, Mount Florida, Glasgow G42 9XW","[{'place_id': 287082716, 'licence': 'Data © Op...",55.8266859,-4.2584184657163,apartments,"(55.8266859, -4.2584184657163)"
1,Full Circle Bakes,"276 Cumbernauld Rd, Glasgow G31 2UL","[{'place_id': 132137206, 'licence': 'Data © Op...",55.8627208,-4.2082693,secondary,"(55.8627208, -4.2082693)"
2,Lily Bakes Cakes,"31 Hyndland St, Partick, Glasgow G11 5QF","[{'place_id': 100429456, 'licence': 'Data © Op...",55.8740428,-4.3015163,residential,"(55.8740428, -4.3015163)"
3,Big Bear Bakery,"21 Dowanhill St, Partick, Glasgow G11 5QR","[{'place_id': 100463582, 'licence': 'Data © Op...",55.8740747,-4.2994975,residential,"(55.8740747, -4.2994975)"
4,Long Short Black,"501 Victoria Rd, Glasgow G42 8RL","[{'place_id': 226016200, 'licence': 'Data © Op...",55.8349688,-4.2653719,tertiary,"(55.8349688, -4.2653719)"
5,Deanston Bakery,"167 Deanston Dr, Shawlands, Glasgow G41 3LP","[{'place_id': 79737616, 'licence': 'Data © Ope...",55.8277017,-4.2822203,bakery,"(55.8277017, -4.2822203)"
6,Akara Bakery,"537 Duke St, Glasgow G31 1DL","[{'place_id': 15176957, 'licence': 'Data © Ope...",55.8585646,-4.2174978,house,"(55.8585646, -4.2174978)"
7,Bertos Brownies,"8 Chancellor St, Glasgow G11 5RQ","[{'place_id': 99498715, 'licence': 'Data © Ope...",55.8717163,-4.2981642,unclassified,"(55.8717163, -4.2981642)"
8,Valaria,"333 Byres Rd, Glasgow G12 8UQ","[{'place_id': 147118923, 'licence': 'Data © Op...",55.876201,-4.2927234,secondary,"(55.876201, -4.2927234)"
9,Chulo's,"936 Argyle St, Finnieston, Glasgow G3 8YJ","[{'place_id': 99774397, 'licence': 'Data © Ope...",55.8596374,-4.2705287,primary,"(55.8596374, -4.2705287)"


In [7]:
place_lat = df1['lat'].astype(float).tolist()

place_lon = df1['lon'].astype(float).tolist()

In [8]:
points = []

for i in range(len(place_lat)):
    points.append([place_lat[i], place_lon[i]])

In [10]:
# Scotland coordinates
scotland_coordinates = (55.8591, -4.2518)

# create empty map zoomed in on Scotland
map = folium.Map(width=800, height=600, location=scotland_coordinates, zoom_start=13)

# add one markercluster per type to allow for individual toggling
bakery = MarkerCluster(name='bakery').add_to(map)

# # add universities to the map
for name in df1.iterrows():
    folium.Marker(location=[float(name[1]['lat']), float(name[1]['lon'])], 
                  popup=name[1]['bakery_name'],
                  icon=folium.Icon(color='green', icon_color='white', icon='star', angle=0, prefix='fa')
                 )\
    .add_to(bakery)

# folium.PolyLine(points).add_to(map)

#plugins.AntPath(route_lats_longs).add_to(map)

# enable toggling of data points
folium.TileLayer('stamenterrain').add_to(map)
folium.raster_layers.TileLayer('Stamen Watercolor').add_to(map)
folium.LayerControl().add_to(map)    

display(map)